In [56]:
# Import findspark and initialize. 
import findspark
findspark.init()

In [57]:
# Import packages
from pyspark.sql import SparkSession
import time

# Create a SparkSession
spark = SparkSession.builder.appName("SparkSQL").getOrCreate()

In [58]:
# 1. Read in the AWS S3 bucket into a DataFrame.
from pyspark import SparkFiles
url = "https://2u-data-curriculum-team.s3.amazonaws.com/dataviz-classroom/v1.1.2/22-big-data/home_sales_revised.csv"
spark.sparkContext.addFile(url)
df = spark.read.csv(SparkFiles.get("home_sales_revised.csv"), sep=",", header=True)
df.show()


+--------------------+----------+----------+------+--------+---------+-----------+--------+------+----------+----+
|                  id|      date|date_built| price|bedrooms|bathrooms|sqft_living|sqft_lot|floors|waterfront|view|
+--------------------+----------+----------+------+--------+---------+-----------+--------+------+----------+----+
|f8a53099-ba1c-47d...|2022-04-08|      2016|936923|       4|        3|       3167|   11733|     2|         1|  76|
|7530a2d8-1ae3-451...|2021-06-13|      2013|379628|       2|        2|       2235|   14384|     1|         0|  23|
|43de979c-0bf0-4c9...|2019-04-12|      2014|417866|       2|        2|       2127|   10575|     2|         0|   0|
|b672c137-b88c-48b...|2019-10-16|      2016|239895|       2|        2|       1631|   11149|     2|         0|   0|
|e0726d4d-d595-407...|2022-01-08|      2017|424418|       3|        2|       2249|   13878|     2|         0|   4|
|5aa00529-0533-46b...|2019-01-30|      2017|218712|       2|        3|       196

In [59]:
# 2. Create a temporary view of the DataFrame.
df.createOrReplaceTempView('home_sales')


In [60]:
# 3. What is the average price for a four bedroom house sold in each year rounded to two decimal places?
spark.sql("""select substring(date,0,4) as year, round(avg(price),2) as price
from home_sales where bedrooms>=4
group by year
order by year""").show(truncate=False)


+----+---------+
|year|price    |
+----+---------+
|2019|362655.25|
|2020|357407.83|
|2021|351185.49|
|2022|350133.71|
+----+---------+



In [61]:
# 4. What is the average price of a home for each year the home was built that have 3 bedrooms and 3 bathrooms rounded to two decimal places?
spark.sql("""select date_built as year_built, round(avg(price),2) as price
from home_sales where bedrooms=4 and bathrooms=3
group by year_built
order by year_built""").show(truncate=False)



+----------+---------+
|year_built|price    |
+----------+---------+
|2010      |287529.97|
|2011      |294656.14|
|2012      |289533.13|
|2013      |291870.89|
|2014      |289935.92|
|2015      |299517.5 |
|2016      |289090.11|
|2017      |289100.17|
+----------+---------+



In [62]:
# 5. What is the average price of a home for each year built that have 3 bedrooms, 3 bathrooms, with two floors,
# and are greater than or equal to 2,000 square feet rounded to two decimal places?
spark.sql("""select date_built as year_built, round(avg(price),2) as price
from home_sales where bedrooms=4 and bathrooms=3 and floors=2 and sqft_living>2000
group by year_built
order by year_built""").show(truncate=False)


+----------+---------+
|year_built|price    |
+----------+---------+
|2010      |289957.62|
|2011      |293021.78|
|2012      |290024.97|
|2013      |293916.19|
|2014      |306202.48|
|2015      |305769.02|
|2016      |298295.66|
|2017      |289215.12|
+----------+---------+



In [63]:
# 6. What is the "view" rating for the average price of a home, rounded to two decimal places, where the homes are greater than
# or equal to $350,000? Although this is a small dataset, determine the run time for this query.

start_time = time.time()

spark.sql("""select round(avg(view),2) as view_rating
from home_sales where price>=350000""").show(truncate=False)
uncached_time = time.time() - start_time
print("--- %s seconds ---" % (uncached_time))

+-----------+
|view_rating|
+-----------+
|32.26      |
+-----------+

--- 0.11902689933776855 seconds ---


In [64]:
# 7. Cache the the temporary table home_sales.
spark.sql("cache table home_sales")

DataFrame[]

In [65]:
# 8. Check if the table is cached.
spark.catalog.isCached('home_sales')

True

In [66]:
# 9. Using the cached data, run the query that filters out the view ratings with average price of greater than $350,000. 
# Determine the runtime and compare it to uncached runtime.

start_time = time.time()

spark.sql("""select round(avg(view),2) as view_rating
from home_sales where price>=350000""").show(truncate=False)
cached_time = time.time() - start_time
print("--- %s seconds for cached table ---" % (cached_time))
print("--- %s seconds for uncached table---" % (uncached_time))


+-----------+
|view_rating|
+-----------+
|32.26      |
+-----------+

--- 0.0780179500579834 seconds for cached table ---
--- 0.11902689933776855 seconds for uncached table---


In [67]:
# 10. Partition the home sales dataset by the date_built field.
df.write.parquet('parquet_home_sales', mode='overwrite')

In [68]:
# 11. Read the formatted parquet data.
p_df=spark.read.parquet('parquet_home_sales')

In [69]:
# 12. Create a temporary table for the parquet data.
p_df.createOrReplaceTempView('p_home_sales')

In [70]:
# 13. Run the query that filters out the view ratings with average price of greater than $350,000 with the parquet DataFrame. 
# Round your average to two decimal places. 
# Determine the runtime and compare it to the cached version. 

start_time = time.time()

spark.sql("""select round(avg(view),2) as view_rating
from p_home_sales where price>=350000""").show(truncate=False)
parquet_time = time.time() - start_time
print("--- %s seconds for parquet table---" % (parquet_time))
print("--- %s seconds for cached table ---" % (cached_time))
print("--- %s seconds for uncached table---" % (uncached_time))

+-----------+
|view_rating|
+-----------+
|32.26      |
+-----------+

--- 0.09402155876159668 seconds for parquet table---
--- 0.0780179500579834 seconds for cached table ---
--- 0.11902689933776855 seconds for uncached table---


In [71]:
# 14. Uncache the home_sales temporary table.
spark.sql("uncache table home_sales")

DataFrame[]

In [72]:
# 15. Check if the home_sales is no longer cached
spark.catalog.isCached("home_sales")


False